In [1]:
import duckdb
import pandas as pd

In [2]:
conn = duckdb.connect('data/patient_experiment.csv')

In [25]:
conn.execute('SELECT DISTINCT patient FROM "patient_experiment";').df()

,patient
0,1
1,3
2,6
3,7
4,8
5,5
6,9
7,2
8,4


In [15]:
sql_pivot = '''
CREATE TABLE time_series AS
  PIVOT "data/patient_experiment.csv"
  ON patient
  USING
    first(temperature) AS temperature,
    first(dose) AS dose
  GROUP BY date;
'''

In [16]:
conn.execute(sql_pivot)

In [18]:
conn.execute('SELECT date FROM time_series;').df()

,date
0,2012-10-02 14:00:00
1,2012-10-02 11:00:00
2,2012-10-02 10:00:00
3,2012-10-02 13:00:00
4,2012-10-02 12:00:00
5,2012-10-02 15:00:00
6,2012-10-02 16:00:00


In [23]:
conn.execute('show tables;').df()

,name
0,file
1,patient_experiment
2,time_series


In [24]:
conn.sql("""
    SELECT table_schema, table_name, table_type
    FROM information_schema.tables
    WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
""").df()

,table_schema,table_name,table_type
0,main,time_series,BASE TABLE
1,main,file,VIEW
2,main,patient_experiment,VIEW


In [28]:
conn.sql('''dESCRIBE patient_experiment;''')

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ column0     │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ patient     │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ dose        │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ date        │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │
│ temperature │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘